# Python 0-30
----------------------------------

Welcome back to Python 0 to 30 for scientists. 

Part 2 is designed to take 1 to 2 work days.

At the end of this tutorial you will have a grasp of how to organize your workflows to enable more complicated and realistic Python analysis.

## Part 2 - Using Python Packages
-----------------------------

In this section we will first learn how to create your own Python package and then how to import and use other popular packages.

Packages allow you to break up complicated unwieldy workflows into smaller subtask building blocks that you pull together into a smaller, more readable code block.

Perhaps you are already familiar with importing packages into your workflow. In IDL people pass around files that contain unique user-written functions -  which reduces redundant work between scientists, but in Python, package managers keep track of what version of those functions you are using. Matlab also has packages that you can pay extra money to install and use - again Python is free! 

## Part 2a - First Python Package
---------------------------------------

In this section we will learn how to move common code blocks into Python modules and packages that
you can import into your complicated analysis methods easier to write and read.

Open a terminal to begin and make sure you are in the NCAR_python_tutorial_2020 directory.

1. [bash] Make a copy of your first script with a new name:
   
   ```bash
   $ cp windchillcomp.py heatindexcomp.py
   ```

2. [git] And add and commit this new file.
   (`git commit -m "Copying first script to start second"`)

3. [python] Now you will compute the Heat Index.

    Like windchill, which is a measure of how much colder the weather feels to the human body due to windspeed, heat index is a measure of how much hotter the weather feels to the human body due to humidity. 
    
    The Rothfusz formula for heat index is:

    $$ HI = a + (b * t) + (c * h) + (d * t * h) + (e * t^2) + (f * h^2) + (g * t^2 * h) + (h * t * h^2) + (i * t^2 * h^2)$$
    

    where *HI* is the Heat Index, *t* represents Temperature is in degrees F, *h* represents Humidity in %, *a* = -42.379, *b* = 2.04901523, *c* = 10.14333127, *d* = 0.22475541, *e* = 0.00683783, *f* = 0.05481717, *g* = 0.00122874, *h* = 0.00085282, and *i* = 0.00000199. The Roothfusz regression is not valid for extreme temperature or hummidity conditions.

    Replace the `compute_windchill` function in your new file with a `compute_heatindex` function:
   
    ```python
    # Compute the heat index
    def compute_heatindex(temp, hum):
        a = -42.379
        b = 2.04901523
        c = 10.14333127
        d = 0.22475541
        e = 0.00683783
        f = 0.05481717
        g = 0.00122874
        h = 0.00085282
        i = 0.00000199

        h = hum / 100

        hi = a + (b * temp) + (c * h) + (d * temp * h) 
            + (e * temp^2) + (f * h^2) + (g * temp^2 * h) 
            + (h * temp * h^2) + (i * temp^2 * h^2)
        return hi
    ```
   
   Change the columns we read from the data file to get humidity and the correct heat index values:
   
    ```python
    columns = {'date': 0, 'time': 1, 'tempout': 2, 'humout': 5,
              'heatindex': 13}
    ```
   
   and specify in our types `dict` that we want `floats`:
   
    ```python
    types = {'tempout': float, 'humout': float, 'heatindex': float}
    ```

   Update the rest of the script to match
   
   ```python
    # Let's actually compute the heat index
    heatindex = []
    for temp, hum in zip(data['tempout'], data['humout']):
        heatindex.append(compute_heatindex(temp, hum))

    # Output comparison of data
    print('                ORIGINAL  COMPUTED')
    print(' DATE    TIME  HEAT INDX HEAT INDX DIFFERENCE')
    print('------- ------ --------- --------- ----------')
    for date, time, hi_orig, hi_comp in zip(data['date'], data['time'], data['heatindex'], heatindex):
        print(f'{date} {time:>6} {hi_orig:9.6f} {hi_comp:9.6f} {hi_orig-hi_comp:10.6f}')
   ```
   
   Run this script and see the results. 

   So in this step you have quickly revisited all of the concepts from part 1 but no new concepts were introduced. Please reach out if this was too quick.

4. [git] Stage and commit this new script.
   (`git commit -m "Updating new heat index script"`)

5. [bash] Now, you have two scripts that do very similar things. In fact, all of the data reading and parsing code is duplicated! And the output is similarly formatted, too.  
   
   Create a new file called `readdata.py`:
   
    ```python
    def read_data(columns, types={}, filename="data/wxobs20170821.txt"):
        """
        Read data from CU Boulder Weather Station data file
       
        Parameters:
            columns: A dictionary of column names mapping to column indices
            types: A dictionary of column names mapping to types to which
               to convert each column of data
            filename: The string path pointing to the CU Boulder Weather
               Station data file
        """
        # Initialize my data variable
        data = {}
        for column in columns:
            data[column] = []

        # Read and parse the data file
        with open(filename, 'r') as datafile:

             # Read the first three lines (header)
            for _ in range(3):
                datafile.readline()

            # Read and parse the rest of the file
            for line in datafile:
                split_line = line.split()
                for column in columns:
                    i = columns[column]
                    t = types.get(column, str)
                    value = t(split_line[i])
                    data[column].append(value)

        return data
    ```
   
   In doing so, you have moved your common code, the function for reading the file from both the windchill and heatindex file to its own file. This is called code refactoring. 

    And you've made some changes to the function, most notably we've introduced keyword arguments - when you see types={} it means that types is presumed to be an empty dictionary if you don't specify it (and so you don't have to specify it every time you call the function when this keyword isn't relevant). 
    
    Similarly you've specified a filename that is used as long as the user doesn't specify a different file. Keyword arguments also can be called in any order.

    Another change to youur function is the inclusion of a docstrings. The "Read data from CU Boulder Weather Station data file" describing the utility of the function and the list of parameters are standard information included in a docstring.

    This new file is a *module*. Modules are simply files containing Python code, meant to be called up within a different Python script.

6. [git] Stage and commit this new file
   (`git commit -m "Adding new readdata module"`)

7. [python] Amend your two Python scripts by first deleting the equivalent read-file code in them.
   
   Then, add the following import statement to the top of each script:
   
   ```python
   from readdata import read_data   
   ```
   
   And finally, after the initializations of the `columns` and `types` variables, replace the deleted code with a function call.
   
   ```python
   # Read data from file
   data = read_data(columns, types=types)
   ```
   
   Test out both of these scripts to make sure they still work!
   
   In python you can call up functionality from scripts outside of your active script using the 'import' statement. Here we import our `read_data` function from the `readdata` module. And now we can call up the function from these scripts. This looks cleaner!

8. [git] Do a `git status` now.  Do you notice something new?  Running our new scripts created the `__pycache__` directory.  
   
    **What is `__pycache__?**
    When you run a python program with an `import` command, Python learns that you have written code that you may call again. The interpreter compiles your scripts to bytecode and stores them in a cache, making your scripts run a little faster next time. As a user can for the most part ignore this new folder. If you change or delete your scripts they will be recompiled and reappear in this folder.

   However, you *don't* want to add this directory to our project repository, so before you commit anything, tell git to ignore it!
   
   Create a new file (in the top-level directory of your project) called `.gitignore` with the following contents:
   
   ```text
   __pycache__/
   ```
   
   Do another `git status`.  What do you see?
   
   Now, instead of `__pycache__` being listed as "untracked", you see `.gitignore` being listed as "untracked"...and no mention of `__pycache__`.
   
   Stage and commit the new `.gitignore` file. 
   (`git commit -m "Ignoring pycache"`)
   
   Finally, do another `git status`.  Notice that the edits you made to your two scripts have still not been committed tothe project repository!  This is because they weren't staged before the last commit.
   

9. [git] Stage *both files* and commit all new changes in one commit
   (`git commit -m "Refactor scripts to use new module"`)

10. [python] There is still some duplicated code in the two scripts.  Namely, the final output code is mostly the same. 
   
   Create another module file called `printing.py` with the following contents:
   
    ```python
    def print_comparison(name, date, time, original_data, computed_data):
        """
        Print a comparison of two timeseries (original and computed)

        Parameters:
           name: A string name for the data being compared. (Limited
               to 9 characters in length)
           date: List of strings representing the dates for each data element
           time: List of strings representing time of day for each data element
           original_data: List of original data (floats)
           computed_data: List of computed data (floats)
        """
        print(f'                ORIGINAL  COMPUTED')
        print(f' DATE    TIME  {name.upper():>9} {name.upper():>9} DIFFERENCE')
        print(f'------- ------ --------- --------- ----------')
        for date, time, orig, comp in zip(date, time, original_data, computed_data):
            print(f'{date} {time:>6} {orig:9.6f} {comp:9.6f} {orig-comp:10.6f}')
    ```

   the only new functionality shown here is `string.upper()`, which capitalzes all lower case letters in a string
   
   Edit the two scripts to use this new module (similar methods to step #7) Test your results.
   
   Try to do this on your own first, but if you are getting error messages the solution looks like:
  
   -- 1.  Add the `from printing import print_comparison` line to the top of each script
   
   -- 2. Replace the printing output section at the bottomof each script with:
   
   ```python
   print_comparison('WINDCHILL', data['date'], data['time'], data['windchill'], windchill)
   ```
   
   or
   
   ```python
   print_comparison('HEAT INDX', data['date'], data['time'], data['heatindex'], heatindex)
   ```

11. [git] Stage all changes and commit 
(`git commit -m "Creating printing module"`)

12. [python] You now have 2 different modules related to the same project.  It is best practice to separate different functions into different modules depending upon the kind of functionality they represent.  In this case, you've separated out the concepts of "data input" and "printing output" into different modules.
    
    Do the same thing with the computation functions, `compute_windchill` and `compute_heatindex`.
    
    Move these functions into a new module called `computation.py`, and modify the scripts to use this new module.  Remember to add docstrings!
    
    Try to do this on your own first!!
    
    Your new `computation.py` module should look similar to the following:
   
    ```python
    def compute_windchill(temp, windspeed):
        """
        Compute the wind chill factor given the 
        temperature and wind speed

        NOTE: This computation is valid only for 
        temperatures between -45F and +45F and for 
        wind speeds between 3 mph and 60 mph.

        Parameters:
            temp: The temperature in units of F
            windspeed: The wind speed in units of mph
        """

        a = 35.74
        b = 0.6215
        c = 35.75
        d = 0.4275

        v16 = windspeed ** 0.16
        wci = a + (b * temp) - (c * v16) + (d * temp * v16)
        return wci


    def compute_heatindex(temp, hum):
        """
        Compute the heat index given the temperature and the humidity

        Parameters:
           temp: The temperature in units of F
           hum: The relative humitidy in units of %
       """

        a = -42.379
        b = 2.04901523
        c = 10.14333127
        d = 0.22475541
        e = 0.00683783
        f = 0.05481717
        g = 0.00122874
        h = 0.00085282
        i = 0.00000199

        rh = hum / 100
    `
        hi = a + (b * temp) + (c * rh) + (d * temp * rh) 
        + (e * temp^2) + (f * rh^2) + (g * temp^2 * rh) 
        + (h * temp * rh^2) + (i * temp^2 * rh^2)
        return hi
    ```
   
    And then modified the scripts accordingly (import your function from `computation.py` and call your function).

13. [git] Stage and commit everything 
    (`git commit -m "Creating computation module"`)

14. [python] Now, you've got quite a few Python files in the main directory.Which ones are scripts?  Which ones are modules meant to be imported?
   
    Typically, you should group all of the modules meant for import only into another directory called a *package*.  A *package* is a directory, often with an `__init__.py` file inside it.
    
    Create a new directory called `mysci` and create an empty file in it called `__init__.py`:
    
    ```bash
    $ mkdir mysci
    $ cd mysci
    $ touch __init__.py
    $ cd ..
    ```
    
    Then, move 3 modules into this package:
    
    ```bash
    $ git mv readdata.py mysci/
    $ git mv printing.py mysci/
    $ git mv computation.py mysci/
    ```
    
    Then, let's modify the import statements at the top of our two scripts so that the modules are automatically imported from the new package:
    
    ```python
    from mysci.readdata import read_data
    from mysci.printing import print_comparison
    from mysci.computation import compute_heatindex
    ```

15. [git] Stage everything (don't forget the `__init__.py` file!) and commit 
    (`git commit -m "Creating mysci package"`)
   
    Our commits are getting bigger, but that's okay.  Each commit corresponds to a *single* (conceptually) change to the codebase.
   
    With this last change, our project should look like this (ignoring the `__pycache__` directories):
   
    ```text
    NCAR_python_tutorial_2020/
    
        data/
            wxobs20170821.txt
            
        mysci/
            __init__.py
            readdata.py
            printing.py
            computation.py
            
        heatindexcomp.py
        windchillcomp.py
   ```

16. [python] As a brief aside -- look at the use of the computation functions in these scripts.  
    
    In the case of the wind chill factor computation, it looks like this:
   
    ```python
    windchill = []
    for temp, windspeed in zip(data['tempout'], data['windspeed']):
        windchill.append(compute_windchill(temp, windspeed))
    ```
    
    This divides the initialization of the `windchill` variable as an empty `list` from the "filling" of that `list` with computed values.
    
    Python gives you some shortcuts to doing this via a concept called  "comprehensions", which are ways of initializing containers (`list`s, `dict`s, etc.) with an *internal loop*.  For example, we could have written the previous 3 lines in the form of a "one-liner" like so:
    
    ```python
    windchill = [compute_windchill(t, w) for t, w in zip(data['tempout'], data['windspeed'])]
    ```
    
    This is a *list comprehension*, and it initializes the entire list with the computed contents, rather than initializing an empy list and appending values to it after the fact. Computationally, this is actually *more efficient*.
    
    Use list comprehensions to make the computation steps in both of scripts one-liners.

17. [git] Stage and commit changes 
    (`git commit -m "Using list comprehensions"`)

---------------------

That concludes the first lesson of Part 2.

You should now be familiar with code refactoring, modules and packages, using the import statement, keyword arguments, function docstrings, some more f-string formatting options, `__pycache__`, `gitignore`, `__init__`, and list comprehensions.

Please continue to [Part 2b]((z230_p2b.ipynb).